<a href="https://colab.research.google.com/github/KrawczykA/Warsztaty_Badawcze/blob/kingston/Fall_wei%C3%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install lightly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#1. importy (nie mam zielonego pojęcia czy wszystkie potrzebne nie mam tez zielonego pojęcia czy colab się na nie nie obrazi)

In [3]:
import os
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import xml.etree.ElementTree as ET
import shutil
from sklearn.model_selection import train_test_split
import timm
from lightly.loss import NTXentLoss, DINOLoss
from lightly.models.modules.heads import SimCLRProjectionHead, BYOLProjectionHead, BYOLPredictionHead
from lightly.models.utils import update_momentum, deactivate_requires_grad

#2. Przygotowanie zbioru danych (wykonane razy liczba zbiorów danych)

## W tej sekcji chcemy uzyskać 4 datasety:


1.   train_ssl_dataset - do użycia podczas SSL jak nazwa wskazuje
2.   train_dataset - do trenowania projection heada
3.   train_full_dataset - połączone oba powyższe do trenowania prymitywnego modelu
4.   test_dataset - do ewaluacji zarówno SSL jak i samej klasyfikacji

w przypadku każdego datasetu będą potrzebne różne tranformacje (jak w notebooku week 5 Part II)

Biorąc pod uwagę terminy do poniedziałku wieczór kod robiący te datasety dla każdego rozważanego zbioru musi być gotowy.



In [ ]:

def create_dataset(set_name, SSL_proportion, train_transform, train_full_transform, test_transform, path_to_data, seed=42, download=True):
    if set_name == 'CIFAR10':
        train_ssl_dataset = torchvision.datasets.CIFAR10(root=path_to_data, train=True, transform=train_transform, download=download)
        train_full_dataset = torchvision.datasets.CIFAR10(root=path_to_data, train=True, transform=train_full_transform, download=download)
        test_dataset = torchvision.datasets.CIFAR10(root=path_to_data, train=False, transform=test_transform, download=download)
    elif set_name == 'CIFAR100':
        train_ssl_dataset = torchvision.datasets.CIFAR100(root=path_to_data, train=True, transform=train_transform, download=download)
        train_full_dataset = torchvision.datasets.CIFAR100(root=path_to_data, train=True, transform=train_full_transform, download=download)
        test_dataset = torchvision.datasets.CIFAR100(root=path_to_data, train=False, transform=test_transform, download=download)
    #a bit different logic you have to have dataset already downloaded and restructured
    else:
        train_path = os.path.join(path_to_data, 'imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train')
        test_path = os.path.join(path_to_data, 'imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val_restructured')
        train_ssl_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=train_transform)
        train_full_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=train_full_transform)
        test_dataset = torchvision.datasets.ImageFolder(root=test_path, transform=test_transform)

    targets = np.array([y for _, y in train_full_dataset])
    SSL_indices, classification_indices = train_test_split(
        np.arange(len(targets)),
        test_size=1-SSL_proportion,
        random_state=seed,
        stratify=targets
    )
    train_dataset = torch.utils.data.Subset(train_full_dataset, classification_indices)
    train_ssl_dataset = torch.utils.data.Subset(train_ssl_dataset, SSL_indices)
    print("Length of entire train dataset: ", len(train_full_dataset))
    print("Length of SSL train dataset: ", len(train_ssl_dataset))
    print("Length of classification train dataset: ", len(train_dataset))
    print("Length of test dataset: ", len(test_dataset))
    return train_full_dataset, train_ssl_dataset, train_dataset, test_dataset

# funkcja do restrukturyzacji bazowego pobrania imagenetu (zmodyfikuj bazowe ścieżki jesli trzymasz dane gdzie indziej)

def restructure_val_dir(imagenet_val_restructured_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val_restructured",
                        imagenet_val_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val",
                         imagenet_train_dir = "data/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train",
                         annotations_dir = "data/imagenet-object-localization-challenge/ILSVRC/Annotations/CLS-LOC"):
    if os.path.exists(imagenet_val_restructured_dir):
        print("Restructured validation directory already exists.")
        return imagenet_val_restructured_dir

    print("Restructuring validation directory...")

    # Get the class mapping from training directory
    classes = [d.name for d in os.scandir(imagenet_train_dir) if d.is_dir()]

    # Create class directories in the restructured validation directory
    for class_name in classes:
        os.makedirs(os.path.join(imagenet_val_restructured_dir, class_name), exist_ok=True)

    # Function to extract class ID from XML annotation file
    def get_class_from_annotation(xml_path):
        tree = ET.parse(xml_path)
        root = tree.getroot()
        # Get the first object's name (class)
        obj = root.find('object')
        if obj is not None:
            return obj.find('name').text
        return None

    # Process each validation image
    for img_name in os.listdir(imagenet_val_dir):
        if not img_name.endswith('.JPEG'):
            continue

        # Get base name without extension
        base_name = os.path.splitext(img_name)[0]

        # Find corresponding annotation file
        xml_path = os.path.join(annotations_dir, base_name + '.xml')

        if os.path.exists(xml_path):
            class_name = get_class_from_annotation(xml_path)
            if class_name and class_name in classes:
                # Copy image to the appropriate class directory
                src_path = os.path.join(imagenet_val_dir, img_name)
                dst_path = os.path.join(imagenet_val_restructured_dir, class_name, img_name)
                shutil.copy(src_path, dst_path)

    print("Validation directory restructured successfully.")
    return imagenet_val_restructured_dir

# 3. Przygotowanie pipelinów treningowych

Chodzi o stworzenie klas pochodnych pl.LightningModule dla wszystkich badanych architektur i dla ogólnego modelu klasyfikującego (który będzie montowany z modelów SSLowych i classification heada),
to wystarczy zrobić tylko raz. Proponuję aby każdy model wytrenować biorąc backbone z resnet18 (jak w notebooku week5), backbone ze struktury resnet18 (bez wyuczonych parametrów), backbone w postaci jakiejś prymitywnej architektury w stylu używanej na neuronkach. Tym sposobem mamy 3 razy więcej wykresików do potencjalnego wstawienia do raportu.

**Uwaga** przy trenowaniu modeli SSL, które nie mają pretext tasku jako klasyfikacji, warto (trzeba) wymyślić jakąś metrykę do ewaluacji w notebooku *copy_of_week5* jest to zrobione jako porównywanie z k-meanes metrykami ARI i NMI. Można wymyślić coś innego można zostawić to.

# 4. Trenowanie - szacowany czas pracy człowieka niewiele jeśli nic nie będzie zabugowane czas obliczeń zasadniczo nie wiadomo.

 W skład tego będzie wchodzić trenowanie SSL (raz) trenowanie klasyfikatora z zamrożoną częścią SSL-ową (dwa) i trenowanie klasyfikatora wraz z częścią SSL-ową (trzy). W schemat raz dwa trzy nie wlicza się model prymitywny trenowany tylko raz.

 Ważne z punktu widzenia raportowego będą wykresiki metryk z procesu uczenia, w tytułach wykresów warto zapisywać końcowe wartości metryk chyba, że ktoś ma inny foolproof sposób przechowywania.

 Warto byłoby (niewiadomo czy czas pozwoli) przeczesać choć niewielką siatkę hiperparametrów w stylu lr = [0.01, 0.001, 0.0001], num_epochs = [5, 15, 30] (zachowajmy stałą liczbę epok 10 dla uczenia classification heada - w tej sytuacji model prymitywny uczyłby się [15, 25, 40] epok). Do tego z trzy różne wartości innego hiperparametru w modelach które takie mają (np. temperatura softmaxa w DINO)

 Trzeba uważać tutaj na nazewnictwo zmiennych bo zaskakująco szybko kończy się słownictwo i łatwo o nadpisanie czegoś co się trenowało godzinę (nie mam zielonego pojęcia skad ta uwaga).

 Na koniec tego kroku będziemy chcieli mieć zmienne nie wiem do końca jakiego typu ale takiego żeby kod z *Part XIV* w *week_5.ipnyb* mógł być na nich odpalony (wymaganie interfacowe). W *copy_of_week5* działają  jak można zobaczyć różne typy.

# 5. PCA dla każdego zbioru danych

Autodeksryptywne mam nadzieję

# 6. Porównanie wyników

Zakładając, że mamy modele w postaci w z punktu 4. to jest to już na szczęście czysty copypasting z *week5.ipynb*

# 7. Raport

w wiadomym notebooku są już piękne pytania raportowe, na które powinno być łatwo odpowiedzieć zakładając, że mamy wszystkie obrazki